In [48]:
from langchain.llms.base import LLM
from typing import List, Optional, Dict, Any
from yandex_gpt import YandexGPTConfigManagerForAPIKey, YandexGPT
from env_vars import *

config_manager = YandexGPTConfigManagerForAPIKey(
    model_type="yandexgpt",
    catalog_id=CATALOGUE_ID,
    api_key=API_KEY
)

class YandexGPTWrapper(LLM):
    # def __init__(self, model_type: str, catalog_id: str, api_key: str):
    #     """
    #     Initializes the YandexGPT wrapper with the given configuration.

    #     Args:
    #         model_type (str): The model type (e.g., "yandexgpt").
    #         catalog_id (str): The catalog ID.
    #         api_key (str): The API key for accessing the Yandex GPT API.
    #     """
    config_manager: YandexGPTConfigManagerForAPIKey 
    yandex_gpt: YandexGPT = YandexGPT(config_manager=config_manager)
        
    @property
    def _llm_type(self) -> str:
        return "yandex_gpt_wrapper_because_the_langchain_yandexgpt_wont_let_authorize"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """
        Calls the YandexGPT model with the given prompt and returns the completion.

        Args:
            prompt (str): The input prompt for the model.
            stop (Optional[List[str]]): Optional stop tokens. Not used in this implementation.

        Returns:
            str: The completion text returned by the YandexGPT model.
        """
        messages = [{'role': 'user', 'text': prompt}]
        completion = self.yandex_gpt.get_sync_completion(messages=messages)
        return completion

    @property
    def identifying_params(self) -> Dict[str, Any]:
        """
        Returns the identifying parameters for the model instance.

        Returns:
            Dict[str, Any]: The parameters used to initialize the model.
        """
        return {
            "model_type": self.config_manager.model_type,
            "catalog_id": self.config_manager.catalog_id,
            "api_key": self.config_manager.api_key
        }

# Usage example:
# Assuming you have the CATALOGUE_ID and API_KEY already defined
yandex_llm = YandexGPTWrapper(config_manager=config_manager)
# response = yandex_llm.invoke("Я дам тебе вопрос из 'Что? Где? Когда?' и ответ к нему. Тебе нужно сымитировать рассуждения знатоков. Их диалог. Выдели важные подсказки из вопроса, приводи к ним ассоциации. Используй в том числе идеи из поп-культуры мифологии литературы и мемов и популярные образы. Рассуждай вслух, долго и по шагам и в конце приди к ответу. Бог на одном юмористическом рисунке несколько раз называет ЕГО “хорошим мальчиком”. Назовите ЕГО. Ответ, к которому тебе нужно прийти: Цербер.")
# print(response)


In [49]:
import pandas as pd
from langchain.chains import LLMChain
from langchain_community.llms import YandexGPT
from langchain_core.prompts import PromptTemplate
from time import sleep

In [50]:
df = pd.read_csv("test_res_with_bin.csv")

In [51]:
prompt_str = '''
Ты профессиональный игрок в Что? Где? Когда? Ты проверяешь ответы, которые дали команды.

Вопрос: {q}

Правильный ответ: {a}

Ответ, который дала команда: {pred}. 

Тебе нужно сравнить правильный ответ с ответом команды и порассуждать, соответствует ли ответ команды правильному ответу в контексте вопроса. Обращай внимание на форму вопроса (т.е. что именно спрашивают)! 

Сначала рассуждения!
Потом, после рассуждений ты пишешь свой вердикт: #ВЕРНО, если ответ можно зачесть или #НЕВЕРНО, если нельзя.
'''

# prompt = 'say hi'



In [52]:
prompt = PromptTemplate.from_template(prompt_str)

In [78]:
def func(row):
        llm_chain = {"q": lambda _: row['q'], "a": lambda _: row['a'], "pred":lambda _:  row['predict']} | prompt | yandex_llm
        # llm_chain = prompt | yandex_llm
        # print(llm_chain)
        sleep(2)
        return llm_chain.invoke("")
        # return ""

In [79]:
res = func(df.iloc[0])
res

'#НЕВЕРНО\n\nВ контексте вопроса важно не только само действие, но и его цель. Правильный ответ подразумевает внесение порядка в хаос для получения цельной картины, что применимо к собиранию пазла или мозаики. В ответе команды речь идёт о раскладывании пасьянса, однако это действие не подразумевает внесения порядка, так как целью является решение головоломки, а не упорядочивание элементов.'

In [80]:
from tqdm import tqdm
tqdm.pandas()

In [81]:
# df = df.head(10)
df

,Unnamed: 0.1,Unnamed: 0,q,a,predict,f1_score,exact_match,answer_checking,correct
0,485,492,Автор детективов Иэн Рэнкин увлекается неким з...,"собирать пазл, собирать мозаику, складывать па...",Раскладывать пасьянс.,0.000000,0,#НЕВЕРНО\n\nИэн Рэнкин увлекается собиранием п...,False
1,817,828,Некоторые виды мух называют путниками из-за их...,"спутниками, спутники, спутник",расстоянии,0.000000,0,#НЕВЕРНО.\n\nКоманда ответила на вопрос неполн...,False
2,30,33,Хави Повес активно продвигает концепцию плоско...,"астронавт, космонавт",Космонавт,0.666667,0,#ВЕРНО\nОтвет команды «Космонавт» можно зачест...,True
3,551,560,Борющаяся с музыкальными корпорациями группа C...,троянский конь,Дон Кихот.,0.000000,0,"#НЕВЕРНО\n\nОтвет команды нельзя зачесть, поск...",False
4,96,100,Голландский шахматист Мануэль Босбом выиграл с...,"дырка, дыра",Надпись.,0.000000,0,"#НЕВЕРНО\n\nОтвет команды не подходит, посколь...",False
...,...,...,...,...,...,...,...,...,...
395,289,294,"Василий Рóзанов писал, что Наталья Нóрдман пог...",пылесос,Каспийское море.,0.000000,0,#НЕВЕРНО\nРассуждения: Василий Розанов сравнив...,False
396,238,243,Математик семнадцатого века Джон ВАллис активн...,"курица, куриная лапа, лапа курицы, лапка куриц...",Какофония.,0.000000,0,#НЕВЕРНО\nРассуждения: ответ команды «Какофони...,False
397,0,0,В одном из эпизодов мультфильма Криса Батлера ...,"сэр, сир, sir",Сэр.,0.500000,0,"#ВЕРНО\n\nОтвет команды «сэр» можно зачесть, т...",True
398,832,843,В стихотворении под названием «ОН» Дэвид Лоуре...,"комары, комар, комарик",Элли,0.000000,0,"#НЕВЕРНО\n\nОтвет команды не подходит, посколь...",False


In [82]:
df['answer_checked_2'] = df.progress_apply(func, axis=1)
df

 76%|███████▌  | 302/400 [29:42<09:38,  5.90s/it]


Exception: Failed to send sync request, status code: 429

In [83]:
df

,Unnamed: 0.1,Unnamed: 0,q,a,predict,f1_score,exact_match,answer_checking,correct
0,485,492,Автор детективов Иэн Рэнкин увлекается неким з...,"собирать пазл, собирать мозаику, складывать па...",Раскладывать пасьянс.,0.000000,0,#НЕВЕРНО\n\nИэн Рэнкин увлекается собиранием п...,False
1,817,828,Некоторые виды мух называют путниками из-за их...,"спутниками, спутники, спутник",расстоянии,0.000000,0,#НЕВЕРНО.\n\nКоманда ответила на вопрос неполн...,False
2,30,33,Хави Повес активно продвигает концепцию плоско...,"астронавт, космонавт",Космонавт,0.666667,0,#ВЕРНО\nОтвет команды «Космонавт» можно зачест...,True
3,551,560,Борющаяся с музыкальными корпорациями группа C...,троянский конь,Дон Кихот.,0.000000,0,"#НЕВЕРНО\n\nОтвет команды нельзя зачесть, поск...",False
4,96,100,Голландский шахматист Мануэль Босбом выиграл с...,"дырка, дыра",Надпись.,0.000000,0,"#НЕВЕРНО\n\nОтвет команды не подходит, посколь...",False
...,...,...,...,...,...,...,...,...,...
395,289,294,"Василий Рóзанов писал, что Наталья Нóрдман пог...",пылесос,Каспийское море.,0.000000,0,#НЕВЕРНО\nРассуждения: Василий Розанов сравнив...,False
396,238,243,Математик семнадцатого века Джон ВАллис активн...,"курица, куриная лапа, лапа курицы, лапка куриц...",Какофония.,0.000000,0,#НЕВЕРНО\nРассуждения: ответ команды «Какофони...,False
397,0,0,В одном из эпизодов мультфильма Криса Батлера ...,"сэр, сир, sir",Сэр.,0.500000,0,"#ВЕРНО\n\nОтвет команды «сэр» можно зачесть, т...",True
398,832,843,В стихотворении под названием «ОН» Дэвид Лоуре...,"комары, комар, комарик",Элли,0.000000,0,"#НЕВЕРНО\n\nОтвет команды не подходит, посколь...",False


In [ ]:
df['correct_2'] = df['answer_checked_2'].apply(lambda x: '#ВЕРНО' in x)

In [ ]:
df['correct_2'].sum()

3

In [ ]:
df['correct'].sum()

3